### Q1. Install MLflow

To get started with MLflow you'll need to install the MLflow Python package.

For this we recommend creating a separate Python environment, for example, you can use conda environments, and then install the package there with pip or conda.

Once you installed the package, run the command mlflow --version and check the output.

What's the version that you have?

In [2]:
!mlflow --version

mlflow, version 2.22.0


### Q2. Download and preprocess the data

We'll use the Yellow Taxi Trip Records dataset to predict the duration of each trip.

Download the data for January, February and March 2023 in parquet format from here.

Use the script preprocess_data.py located in the folder homework to preprocess the data.

The script will:

    load the data from the folder <TAXI_DATA_FOLDER> (the folder where you have downloaded the data),
    fit a DictVectorizer on the training set (January 2023 data),
    save the preprocessed datasets and the DictVectorizer to disk.

Your task is to download the datasets and then execute this command:

python preprocess_data.py --raw_data_path <TAXI_DATA_FOLDER> --dest_path ./output

Tip: go to 02-experiment-tracking/homework/ folder before executing the command and change the value of <TAXI_DATA_FOLDER> to the location where you saved the data.

How many files were saved to OUTPUT_FOLDER?

In [17]:
import os
import pickle
import click
import pandas as pd
import sys
from click.testing import CliRunner

from sklearn.feature_extraction import DictVectorizer


def dump_pickle(obj, filename: str):
    with open(filename, "wb") as f_out:
        return pickle.dump(obj, f_out)


def read_dataframe(filename: str):
    df = pd.read_parquet(filename)

    df['duration'] = df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)
    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)

    return df


def preprocess(df: pd.DataFrame, dv: DictVectorizer, fit_dv: bool = False):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    dicts = df[categorical + numerical].to_dict(orient='records')
    if fit_dv:
        X = dv.fit_transform(dicts)
    else:
        X = dv.transform(dicts)
    return X, dv


@click.command()
@click.option(
    "--raw_data_path",
    help="Location where the raw NYC taxi trip data was saved"
)
@click.option(
    "--dest_path",
    help="Location where the resulting files will be saved"
)
def run_data_prep(raw_data_path: str, dest_path: str, dataset: str = "yellow"):
    # Load parquet files
    df_train = read_dataframe(
        os.path.join(raw_data_path, f"{dataset}_tripdata_2023-01.parquet")
    )
    df_val = read_dataframe(
        os.path.join(raw_data_path, f"{dataset}_tripdata_2023-02.parquet")
    )
    df_test = read_dataframe(
        os.path.join(raw_data_path, f"{dataset}_tripdata_2023-03.parquet")
    )

    # Extract the target
    target = 'duration'
    y_train = df_train[target].values
    y_val = df_val[target].values
    y_test = df_test[target].values

    # Fit the DictVectorizer and preprocess data
    dv = DictVectorizer()
    X_train, dv = preprocess(df_train, dv, fit_dv=True)
    X_val, _ = preprocess(df_val, dv, fit_dv=False)
    X_test, _ = preprocess(df_test, dv, fit_dv=False)

    # Create dest_path folder unless it already exists
    os.makedirs(dest_path, exist_ok=True)

    # Save DictVectorizer and datasets
    dump_pickle(dv, os.path.join(dest_path, "dv.pkl"))
    dump_pickle((X_train, y_train), os.path.join(dest_path, "train.pkl"))
    dump_pickle((X_val, y_val), os.path.join(dest_path, "val.pkl"))
    dump_pickle((X_test, y_test), os.path.join(dest_path, "test.pkl"))



if __name__ == '__main__':
    sys.argv = ['run_data_prep', 
            '--raw_data_path', '/home/pce/Downloads/taxi_folder/', 
            '--dest_path', '/home/pce/Downloads/taxi_output/']

    try:
        run_data_prep()
    except SystemExit as e:
        print("Exited with code:", e.code)


Exited with code: 0


### Q3. Train a model with autolog

We will train a RandomForestRegressor (from Scikit-Learn) on the taxi dataset.

We have prepared the training script train.py for this exercise, which can be also found in the folder homework.

The script will:

    load the datasets produced by the previous step,
    train the model on the training set,
    calculate the RMSE score on the validation set.

Your task is to modify the script to enable autologging with MLflow, execute the script and then launch the MLflow UI to check that the experiment run was properly tracked.

Tip 1: don't forget to wrap the training code with a with mlflow.start_run(): statement as we showed in the videos.

Tip 2: don't modify the hyperparameters of the model to make sure that the training will finish quickly.

What is the value of the min_samples_split parameter:

    2
    4
    8
    10


In [23]:
import os
import pickle
import click
import mlflow

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

mlflow.set_tracking_uri('http://localhost:5000')
mlflow.set_experiment("NycTaxiTripExperiment")
mlflow.sklearn.autolog(log_input_examples=False)

def load_pickle(filename: str):
    with open(filename, "rb") as f_in:
        return pickle.load(f_in)


@click.command()
@click.option(
    "--data_path",
    default="./output",
    help="Location where the processed NYC taxi trip data was saved"
)

def run_train(data_path: str):
    
    with mlflow.start_run(run_name='Random Forest Run'):
        X_train, y_train = load_pickle(os.path.join(data_path, "train.pkl"))
        X_val, y_val = load_pickle(os.path.join(data_path, "val.pkl"))

        rf = RandomForestRegressor(max_depth=10, random_state=0)
        rf.fit(X_train, y_train)
        y_pred = rf.predict(X_val)

        rmse = mean_squared_error(y_val, y_pred, squared=False)


if __name__ == '__main__':
    sys.argv = ['run_train', 
            '--data_path', '/home/pce/Downloads/taxi_output/']

    try:
        run_train()
    except SystemExit as e:
        print("Exited with code:", e.code)
    

2025/05/27 07:28:24 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: Unable to allocate 489. GiB for an array with shape (3009173, 21802) and data type float64
2025/05/27 07:56:23 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during sklearn autologging: When an mlflow-artifacts URI was supplied, the tracking URI must be a valid http or https URI, but it was currently set to file:///home/pce/Downloads/mlruns. Perhaps you forgot to set the tracking URI to the running MLflow server. To set the tracking URI, use either of the following methods:
1. Set the MLFLOW_TRACKING_URI environment variable to the desired tracking URI. `export MLFLOW_TRACKING_URI=http://localhost:5000`
2. Set the tracking URI programmatically by calling `mlflow.set_tracking_uri`. `mlflow.set_tracking_uri('http://localhost:5000')`


Exited with code: 0


/home/pce/anaconda3/envs/duckdb_env/lib/python3.11/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


### Q4. Launch the tracking server locally

Now we want to manage the entire lifecycle of our ML model. In this step, you'll need to launch a tracking server. This way we will also have access to the model registry.

Your task is to:

    launch the tracking server on your local machine,
    select a SQLite db for the backend store and a folder called artifacts for the artifacts store.

You should keep the tracking server running to work on the next two exercises that use the server.

In addition to backend-store-uri, what else do you need to pass to properly configure the server?

    default-artifact-root
    serve-artifacts
    artifacts-only
    artifacts-destination


In [ ]:
# mlflow server \
#   --backend-store-uri sqlite:///mlflow.db \
#   --default-artifact-root ./mlruns_artifacts \
#   --host 0.0.0.0 \
#   --port 5000


In [1]:
import subprocess
import mlflow

mlflow.set_tracking_uri('http://localhost:5000')
subprocess.Popen([
    "mlflow", "server",
    "--backend-store-uri", "sqlite:///mlflow.db",
    "--default-artifact-root", "./mlruns_artifacts",
    "--host", "0.0.0.0",
    "--port", "5000"
])


<Popen: returncode: None args: ['mlflow', 'server', '--backend-store-uri', '...>

[2025-05-27 08:37:05 +0200] [18155] [INFO] Starting gunicorn 21.2.0
[2025-05-27 08:37:05 +0200] [18155] [INFO] Listening at: http://0.0.0.0:5000 (18155)
[2025-05-27 08:37:05 +0200] [18155] [INFO] Using worker: sync
[2025-05-27 08:37:05 +0200] [18157] [INFO] Booting worker with pid: 18157
[2025-05-27 08:37:05 +0200] [18158] [INFO] Booting worker with pid: 18158
[2025-05-27 08:37:05 +0200] [18159] [INFO] Booting worker with pid: 18159
[2025-05-27 08:37:05 +0200] [18164] [INFO] Booting worker with pid: 18164


### Q5. Tune model hyperparameters

Now let's try to reduce the validation error by tuning the hyperparameters of the RandomForestRegressor using hyperopt. We have prepared the script hpo.py for this exercise.

Your task is to modify the script hpo.py and make sure that the validation RMSE is logged to the tracking server for each run of the hyperparameter optimization (you will need to add a few lines of code to the objective function) and run the script without passing any parameters.

After that, open UI and explore the runs from the experiment called random-forest-hyperopt to answer the question below.

Note: Don't use autologging for this exercise.

The idea is to just log the information that you need to answer the question below, including:

    the list of hyperparameters that are passed to the objective function during the optimization,
    the RMSE obtained on the validation set (February 2023 data).

What's the best validation RMSE that you got?

    4.817
    5.335
    5.818
    6.336




In [2]:
import os
import sys
import pickle
import click
import mlflow
import numpy as np
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
from hyperopt.pyll import scope
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import root_mean_squared_error

mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("random-forest-hyperopt")


def load_pickle(filename: str):
    with open(filename, "rb") as f_in:
        return pickle.load(f_in)


@click.command()
@click.option(
    "--data_path",
    default="./output",
    help="Location where the processed NYC taxi trip data was saved"
)
@click.option(
    "--num_trials",
    default=15,
    help="The number of parameter evaluations for the optimizer to explore"
)
def run_optimization(data_path: str, num_trials: int):

    X_train, y_train = load_pickle(os.path.join(data_path, "train.pkl"))
    X_val, y_val = load_pickle(os.path.join(data_path, "val.pkl"))

    def objective(params):
        print(f"Starting trial with params: {params}", flush=True)
        with mlflow.start_run(nested=True):
            rf = RandomForestRegressor(**params)
            rf.fit(X_train, y_train)
            y_pred = rf.predict(X_val)
            rmse = root_mean_squared_error(y_val, y_pred)
            
            mlflow.log_params(params)
            mlflow.log_metric('rmse', rmse)
            
            print(f"Finished trial with rmse: {rmse}", flush=True)
            
            return {'loss': rmse, 'status': STATUS_OK}

    search_space = {
        'max_depth': scope.int(hp.quniform('max_depth', 1, 20, 1)),
        'n_estimators': scope.int(hp.quniform('n_estimators', 10, 50, 1)),
        'min_samples_split': scope.int(hp.quniform('min_samples_split', 2, 10, 1)),
        'min_samples_leaf': scope.int(hp.quniform('min_samples_leaf', 1, 4, 1)),
        'random_state': 42
    }

    rstate = np.random.default_rng(42)  # for reproducible results
    fmin(
        fn=objective,
        space=search_space,
        algo=tpe.suggest,
        max_evals=num_trials,
        trials=Trials(),
        rstate=rstate
    )


if __name__ == '__main__':
    sys.argv = ['run_train', 
            '--data_path', '/home/pce/Downloads/taxi_output/']

    try:
        run_optimization()
    except SystemExit as e:
        print("Exited with code:", e.code)
    

Starting trial with params: {'max_depth': 13, 'min_samples_leaf': 4, 'min_samples_split': 7, 'n_estimators': 43, 'random_state': 42}
Finished trial with rmse: 5.080919235745394                                     
🏃 View run fearless-zebra-63 at: http://127.0.0.1:5000/#/experiments/1/runs/8ed9d2d13f964c62adb34a66b8091b1d

🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1                    

Starting trial with params: {'max_depth': 2, 'min_samples_leaf': 4, 'min_samples_split': 7, 'n_estimators': 22, 'random_state': 42}
Finished trial with rmse: 5.785462172175059                                     
🏃 View run polite-slug-263 at: http://127.0.0.1:5000/#/experiments/1/runs/97cbed6626de48c8bf386b8816787175

🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1                    

Starting trial with params: {'max_depth': 4, 'min_samples_leaf': 3, 'min_samples_split': 3, 'n_estimators': 16, 'random_state': 42}
Finished trial with rmse: 5.254544756747019                  

### Q6.  Promote the best model to the model registry

The results from the hyperparameter optimization are quite good. So, we can assume that we are ready to test some of these models in production. In this exercise, you'll promote the best model to the model registry. We have prepared a script called register_model.py, which will check the results from the previous step and select the top 5 runs. After that, it will calculate the RMSE of those models on the test set (March 2023 data) and save the results to a new experiment called random-forest-best-models.

Your task is to update the script register_model.py so that it selects the model with the lowest RMSE on the test set and registers it to the model registry.

Tip 1: you can use the method search_runs from the MlflowClient to get the model with the lowest RMSE,

Tip 2: to register the model you can use the method mlflow.register_model and you will need to pass the right model_uri in the form of a string that looks like this: "runs:/<RUN_ID>/model", and the name of the model (make sure to choose a good one!).

What is the test RMSE of the best model?

    5.060
    5.567
    6.061
    6.568


In [9]:
import os
import pickle
import click
import mlflow

from mlflow.entities import ViewType
from mlflow.tracking import MlflowClient
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import root_mean_squared_error

HPO_EXPERIMENT_NAME = "random-forest-hyperopt"
EXPERIMENT_NAME = "random-forest-best-models"
RF_PARAMS = ['max_depth', 'n_estimators', 'min_samples_split', 'min_samples_leaf', 'random_state']

mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment(EXPERIMENT_NAME)
mlflow.sklearn.autolog()


def load_pickle(filename):
    with open(filename, "rb") as f_in:
        return pickle.load(f_in)


def train_and_log_model(data_path, params):
    X_train, y_train = load_pickle(os.path.join(data_path, "train.pkl"))
    X_val, y_val = load_pickle(os.path.join(data_path, "val.pkl"))
    X_test, y_test = load_pickle(os.path.join(data_path, "test.pkl"))

    with mlflow.start_run():
        new_params = {}
        for param in RF_PARAMS:
            new_params[param] = int(params[param])

        rf = RandomForestRegressor(**new_params)
        rf.fit(X_train, y_train)

        # Evaluate model on the validation and test sets
        val_rmse = root_mean_squared_error(y_val, rf.predict(X_val))
        mlflow.log_metric("val_rmse", val_rmse)
        test_rmse = root_mean_squared_error(y_test, rf.predict(X_test))
        mlflow.log_metric("test_rmse", test_rmse)


@click.command()
@click.option(
    "--data_path",
    default="./output",
    help="Location where the processed NYC taxi trip data was saved"
)
@click.option(
    "--top_n",
    default=5,
    type=int,
    help="Number of top models that need to be evaluated to decide which one to promote"
)
def run_register_model(data_path: str, top_n: int):

    client = MlflowClient()

    # Retrieve the top_n model runs and log the models
    experiment = client.get_experiment_by_name(HPO_EXPERIMENT_NAME)
    runs = client.search_runs(
        experiment_ids=experiment.experiment_id,
        run_view_type=ViewType.ACTIVE_ONLY,
        max_results=top_n,
        order_by=["metrics.rmse ASC"]
    )
#     for run in runs:
#         train_and_log_model(data_path=data_path, params=run.data.params)

    # Select the model with the lowest test RMSE
    experiment = client.get_experiment_by_name(EXPERIMENT_NAME)
    # best_run = client.search_runs( ...  )[0]
    best_run = client.search_runs(
        experiment_ids=experiment.experiment_id,
        run_view_type=ViewType.ACTIVE_ONLY,
        max_results=1,
        order_by=["metrics.test_rmse ASC"]
    )[0]

    # Register the best model
    # mlflow.register_model( ... )
    model_uri = f"runs:/{best_run.info.run_id}/model"
    mlflow.register_model(
        model_uri=model_uri,
        name="random-forest-regressor"
    )
    print(f"Best model registered from run_id {best_run.info.run_id} with test RMSE: {best_run.data.metrics['test_rmse']}")



if __name__ == '__main__':
    sys.argv = ['run_train', 
            '--data_path', '/home/pce/Downloads/taxi_output/']

    try:
        run_register_model()
    except SystemExit as e:
        print("Exited with code:", e.code)
    
    

Registered model 'random-forest-regressor' already exists. Creating a new version of this model...
2025/05/27 12:47:08 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: random-forest-regressor, version 2


Best model registered from run_id 20b9dd2dc252443085767859124f9a3a with test RMSE: 5.5404828839248905
Exited with code: 0


Created version '2' of model 'random-forest-regressor'.
